In [78]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS  
from tenpy.algorithms.network_contractor import ncon

In [ ]:

def random_L(d: int, Dl: int, Dr: int, seed=None) -> np.ndarray:
    """
    Tensore L_k di shape (d, Dl, Dr).
    Su ogni riga non nulla c’è un solo '1' in colonna casuale.
    Ogni riga (x, i) ha probabilità 1/(Dr+1) di essere tutta zero.
    """
    rng = np.random.default_rng(seed)
    L = np.zeros((d, Dl, Dr), dtype=np.uint8)

    # J[x, i] ∈ {0, ..., Dr} con distribuzione uniforme
    J = rng.integers(0, Dr + 1, size=(d, Dl))
    mask = (J != Dr)  # True dove mettiamo un '1'

    xs, is_ = np.where(mask)
    js = J[xs, is_]
    L[xs, is_, js] = 1

    return L


def random_R(d: int, Dl: int, Dr: int, seed=None) -> np.ndarray:
    """
    Tensore R_k di shape (d, Dl, Dr).
    Su ogni colonna non nulla c’è un solo '1' in riga casuale.
    Ogni colonna (x, j) ha probabilità 1/(Dl+1) di essere tutta zero.
    """
    rng = np.random.default_rng(seed)
    R = np.zeros((d, Dl, Dr), dtype=np.uint8)

    # I[x, j] ∈ {0, ..., Dl} con distribuzione uniforme
    I = rng.integers(0, Dl + 1, size=(d, Dr))
    mask = (I != Dl)

    xs, js = np.where(mask)
    is_ = I[xs, js]
    R[xs, is_, js] = 1

    return R


def random_A1(d: int, D: int, seed=None) -> np.ndarray:
    """Primo sito: shape (d, 1, D). Binario libero."""
    rng = np.random.default_rng(seed)
    # Estraggo in {0, 1}, uniforme, direttamente in uint8
    return rng.integers(0, 2, size=(d, 1, D), dtype=np.uint8)

def wrap_site_tensor(T: np.ndarray):
    """(d, Dl, Dr) -> npc.Array labels ['vL','p','vR'] shape (Dl, d, Dr)."""
    # transpose is a view; np Array creation happens inside TeNPy
    return npc.Array.from_ndarray_trivial(T.transpose(1, 0, 2), labels=['vL', 'p', 'vR'])

def tenpy_sites_and_svs(d: int, right_dims):
    """
    Costruisce i siti TenPy e i singolar values iniziali
    per una MPS con dimensione fisica d e bond-dims right_dims.
    right_dims ha lunghezza N, con N = numero siti.
    """
    N = len(right_dims)
    S = (d - 1) / 2.0
    site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=site)
    sites = lattice.mps_sites()

    # svs: D_0, D_1, ..., D_{N-1}, D_N con D_0 = D_N = 1
    svs = [np.ones(1, dtype=float)]
    svs.extend(np.ones(D, dtype=float) for D in right_dims[:-1])
    svs.append(np.ones(1, dtype=float))

    return sites, svs

In [80]:
def site_tensor_from_Bi(Bi: dict, d: int, *, strict_keys: bool = True):
    """
    Costruisce T[x] = Bi[x] con fallback a zeri. T shape: (d, Dl, Dr).

    - Valida che tutte le matrici abbiano la stessa shape 2D.
    - Se strict_keys=True, richiede chiavi intere in [0, d-1].
    - Usa una 'fast path' quando Bi contiene tutte le chiavi 0..d-1 (stack).
    """
    if not Bi:
        raise ValueError("Bi non può essere vuoto: serve almeno una matrice per fissare (Dl, Dr).")

    # --- keys validation (vectorized) ---
    keys = np.fromiter(Bi.keys(), dtype=int, count=len(Bi))
    if strict_keys:
        if keys.dtype.kind not in "iu":
            raise ValueError("Le chiavi di Bi devono essere intere.")
        if keys.min() < 0 or keys.max() >= d:
            bad = [k for k in Bi.keys() if not (0 <= int(k) < d)]
            raise ValueError(f"Chiavi fuori range [0,{d-1}]: {bad}")

    # --- infer shape and common dtype once ---
    first_key = next(iter(Bi))
    first_M = np.asarray(Bi[first_key])
    if first_M.ndim != 2:
        raise ValueError(f"Bi[{first_key}] deve essere una matrice 2D, shape trovata: {first_M.shape}")
    Dl, Dr = first_M.shape

    # common dtype among provided blocks
    common_dtype = np.result_type(*(np.asarray(M).dtype for M in Bi.values()))

    # --- fast path if complete coverage and canonical ordering ---
    if len(Bi) == d and (not strict_keys or set(Bi.keys()) == set(range(d))):
        # assemble in key order 0..d-1; will raise if shape mismatch
        try:
            T = np.stack([np.asarray(Bi[x], dtype=common_dtype) for x in range(d)], axis=0)
        except KeyError as e:
            # unexpected missing key; fall back to sparse path
            T = None
        else:
            # quick shape check
            if T.shape != (d, Dl, Dr):
                raise ValueError(f"Matrici in Bi non omogenee: shape risultante {T.shape}, atteso {(d, Dl, Dr)}")
            return T, Dl, Dr

    # --- sparse path: fill once with zeros, then write only provided keys ---
    T = np.zeros((d, Dl, Dr), dtype=common_dtype)

    # validate shapes cheaply and assign
    for k, M in Bi.items():
        Mk = np.asarray(M, dtype=common_dtype)
        if Mk.shape != (Dl, Dr):
            raise ValueError(f"Bi[{k}] ha shape {Mk.shape}, atteso {(Dl, Dr)}")
        T[int(k)] = Mk  # missing keys remain 0

    return T, Dl, Dr


def build_mps(B_list, d: int):
    """
    Costruisce un MPS TeNPy dai blocchi per-sito (dizionari Bi).
    - Verifica vincoli di bordo (Dl[0]==1, Dr[N-1]==1) e coerenza dei bond interni.
    - Converte ciascun sito in npc.Array con labels ['vL','p','vR'].
    - Inizializza vettori di Schmidt = 1.
    """
    # validate d una sola volta (cheap branch)
    if not isinstance(d, (int, np.integer)) or d < 1:
        raise ValueError("`d` deve essere intero positivo.")

    N = len(B_list)
    if N == 0:
        raise ValueError("B_list non può essere vuoto.")

    A = []
    right_dims = []
    prev_Dr = None

    for i, Bi in enumerate(B_list):
        T, Dl, Dr = site_tensor_from_Bi(Bi, d, strict_keys=True)

        # Bordo sinistro
        if i == 0:
            if Dl != 1:
                raise ValueError(f"sito {i}: Dl deve essere 1 (bordo sinistro), trovato {Dl}")
        else:
            # Bond interno: Dl(i) deve coincidere con Dr(i-1)
            if Dl != prev_Dr:
                raise ValueError(
                    f"mismatch bond interno tra sito {i-1} e {i}: Dr[{i-1}]={prev_Dr}, Dl[{i}]={Dl}"
                )

        # Bordo destro
        if i == N - 1 and Dr != 1:
            raise ValueError(f"sito {i}: Dr deve essere 1 (bordo destro), trovato {Dr}")

        # Wrap immediato (evita una lista intermedia di tensori)
        A.append(wrap_site_tensor(T))
        right_dims.append(Dr)
        prev_Dr = Dr

    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')


In [81]:
def build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu, *, dtype=np.float64, share_middle=True):
    """
    Build B_list faster by precomputing the 3 state blocks once.
    - share_middle=True: reuse the same dict object for all middle sites (fastest).
      Set False to place shallow copies of the dict (arrays still shared).
      If you need full independence, do deep copies per array (slower; see comment).
    """
    # validate probabilities once
    pmid = 1.0 - pd - pu
    if not (0.0 <= pd <= 1.0 and 0.0 <= pu <= 1.0 and pd + pu <= 1.0):
        raise ValueError("Probabilità non valide: servono pd, pu >= 0 e pd+pu <= 1")

    if N < 2:
        raise ValueError("N deve essere >= 2 (primo e ultimo sito esistono sempre).")

    S_over = S0 / (N + 1.0)

    # vectorize ops and probs across the 3 states: 0=down, 1=mid, 2=up
    ops   = np.array([d_op, m_op, u_op], dtype=dtype)     # (3,)
    probs = np.array([pd,   pmid,  pu],   dtype=dtype)     # (3,)

    # handy combinations (all length-3, one per state)
    TL = ops * probs                                       # top-left: op * p
    TR = (S_over * ops - K) * probs                        # top-right: (S/(N+1)*op - K) * p
    BL = probs                                             # bottom-left: p

    # ---------- Site 1: shape (1,2) ----------
    # Row: [ S_over*op*p , (S_over*op - K)*p ]
    row12 = np.stack([S_over * ops * probs, TR], axis=1)   # (3, 2)
    B1 = {
        0: row12[0][None, :].astype(dtype, copy=False),
        1: row12[1][None, :].astype(dtype, copy=False),
        2: row12[2][None, :].astype(dtype, copy=False),
    }

    # ---------- Middle sites i=2..N-1: shape (2,2) ----------
    # [[ op*p , (S_over*op - K)*p ],
    #  [   p  ,          1        ]]
    one = np.array(1.0, dtype=dtype)
    Bi_template = {
        0: np.array([[TL[0], TR[0]],
                     [BL[0], one  ]], dtype=dtype),
        1: np.array([[TL[1], TR[1]],
                     [BL[1], one  ]], dtype=dtype),
        2: np.array([[TL[2], TR[2]],
                     [BL[2], one  ]], dtype=dtype),
    }

    # ---------- Last site N: shape (2,1) ----------
    # [[ op*p ],
    #  [  p   ]]
    BN = {
        0: np.array([[TL[0]],
                     [BL[0]]], dtype=dtype),
        1: np.array([[TL[1]],
                     [BL[1]]], dtype=dtype),
        2: np.array([[TL[2]],
                     [BL[2]]], dtype=dtype),
    }

    # ---------- Assemble B_list ----------
    B_list = [B1]
    if N > 2:
        if share_middle:
            # fastest: reuse the exact same dict object for all middle sites
            B_list.extend([Bi_template] * (N - 2))
        else:
            # safer: separate dicts pointing to the same arrays (still shared)
            B_list.extend([Bi_template.copy() for _ in range(N - 2)])
            # If you truly need fully independent arrays per site, use:
            # for _ in range(N - 2):
            #     B_list.append({k: v.copy() for k, v in Bi_template.items()})
    B_list.append(BN)
    return B_list


In [82]:
def build_mps_AR(d: int, N: int, D: int, seed=None):
    """
    Crea b(x) con bond dimensione D:
      A1: (d, 1, D)   [NB: original line below uses D=d; see NOTE]
      R2..R_{N-1}: (d, D, D)
      RN: (d, D, 1)
    """
    # controlli basilari
    if N < 2:
        raise ValueError("N deve essere >= 2 per avere A1 e almeno un R.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d); D = int(D)

    # ----- RNG seeds (cheap to compute, avoids repeated branching) -----
    # Keep the same seeding pattern you used: seed, seed+2..seed+(N-1), seed+N
    # (Python int math is cheap; this is just clarity)
    sA = seed
    s_list = [None if seed is None else seed + k for k in range(2, N)]
    sN = None if seed is None else seed + N

    # ----- Tensors -----
    # IMPORTANT NOTE:
    # Your original line is: A1 = random_A1(d, d, seed=seed)  -> shape (d,1,d)
    # That forces Dr0 = d (not D). I keep it for output-compatibility.
    # If you *intended* (d,1,D), change the next line to random_A1(d, D, seed=sA).
    A1 = random_A1(d, d, seed=sA)  # keep output identical to your current code

    # build middle R's via comprehension (fast) — (d, D, D)
    Rs = [random_R(d, Dl=D, Dr=D, seed=s) for s in s_list]

    # last site RN — (d, D, 1)
    RN = random_R(d, Dl=D, Dr=1, seed=sN)

    tensors = [A1, *Rs, RN]

    # wrap TenPy (cheap comprehensions; no semantic changes)
    A = [wrap_site_tensor(T) for T in tensors]
    right_dims = [T.shape[2] for T in tensors]
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')


def build_mps_AR_bond(d: int, N: int, D: int, seed=None):
    """
    Crea un MPS psi con N siti (N dispari) e bond che crescono verso il centro
    come Dl -> min(Dl * d, D) e poi decrescono simmetricamente.

    Tensore sito i (0-based) ha shape (d, Dl_i, Dr_i).
    """
    # controlli basilari
    if N < 3 or N % 2 == 0:
        raise ValueError("N deve essere dispari e >= 3.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d); D = int(D)

    half = N // 2

    # ----- bonds: build left→center once, then mirror -----
    bonds = [1]
    Dl_curr = 1
    for _ in range(1, half + 1):
        Dl_curr = Dl_curr * d
        if Dl_curr > D:
            Dl_curr = D
        bonds.append(Dl_curr)

    bonds = bonds + bonds[::-1]  # symmetric [1,...,center,...,1]
    # N sites => N+1 bonds
    # assert here kept, but using cheap check only
    if len(bonds) != N + 1:
        raise RuntimeError("Costruzione bonds fallita (len != N+1).")

    # ----- seeds for reproducibility (same pattern you used) -----
    sA = seed
    s_list = [None if seed is None else seed + i for i in range(1, N)]  # i = 1..N-1 for the R's

    tensors = [None] * N

    # first site: Dl=1, Dr=bonds[1]
    Dr0 = bonds[1]
    # Keep exact current behavior of your A1 generator:
    # your build_mps_AR_bond already calls random_A1(d, Dr0, seed=seed)
    tensors[0] = random_A1(d, Dr0, seed=sA)

    # sites 1..N-1
    # comprehension with enumerate keeps Python overhead low
    tensors[1:] = [
        random_R(d, Dl=bonds[i], Dr=bonds[i+1], seed=s_list[i-1])
        for i in range(1, N)
    ]

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]
    right_dims = [T.shape[2] for T in tensors]
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')


In [218]:
S0, K, N = 1.0, 0.2, 21
d_op, m_op, u_op = 0.9, 1.0, 3
pd, pu = 0.2, 0.4   

B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)

b = build_mps(B_list, d=3)

psi = build_mps_AR_bond(d=3, N=len(B_list), D=9)

val = b.overlap(psi)
print("sum_x psi(x)b(x) =", val)

sum_x psi(x)b(x) = -26855935.78932268


In [84]:
def contract_left(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i-1)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == 1:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")


    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # A0: (-1, -2,  1)
    # A1: ( 1, -3,  2)
    # A2: ( 2, -4,  3)
    # ...
    # A_{i-2}: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d

    return T

def contract_right(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i,N)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == N:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")

    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # Ai: (-1, -2,  1)
    # Ai+1: ( 1, -3,  2)
    # Ai+2: ( 2, -4,  3)
    # ...
    # AN-1: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d
    return T

UL = contract_left(tensor=psi,i=3,N=len(B_list))
UR = contract_right(tensor=psi,i=3,N=len(B_list))


In [85]:
def tensorial_derivative(psi, b, site):

    N_psi = psi.L
    N_b  = b.L
    if N_psi != N_b:
        raise ValueError(f"psi e b hanno lunghezze diverse: {N_psi} vs {N_b}")
    N = N_psi

    if not (1 <= site <= N):
        raise ValueError(f"site={site} fuori range [1, {N}]")
    
    UL = contract_left(tensor=psi,i=site,N=N)
    UR = contract_right(tensor=psi,i=site,N=N)
    BL = contract_left(tensor=b,i=site,N=N)
    BR = contract_right(tensor=b,i=site,N=N)

    if UL is not None and len(UL.shape) != len(BL.shape):
        raise ValueError(f"Dimensione di UL diversa da BL")
    if UR is not None and len(UR.shape) != len(BR.shape):
        raise ValueError(f"Dimensione di UR diversa da BR")

    if UL is not None:
        left_tensors = [UL,BL]
        left_links = [
            [-1] + [k for k in range(1,site)] + [-2], 
        [-3] + [k for k in range(1,site)] + [-4] 
        ]
        #print('left contraction links',left_links)
        left_contraction = ncon(left_tensors,left_links) 
    ## Il risultato è un tensore di dimensione (1,D_i;1,2)

    if UR is not None:
        right_tensors = [UR,BR]
        right_links = [
            [-1] + [k for k in range(1,N-site+1)] + [-2], 
        [-3] + [k for k in range(1,N-site+1)] + [-4] 
        ]
        #print('right contraction links',right_links)
        right_contraction = ncon(right_tensors,right_links) 
    ## Il risultato è un tensore di dimensione (D_i+1,1;2,1)

    if site > 1 and site < N:
        final_tensors = [left_contraction, b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [2],
            [-5] + [-6] + [2] + [-7]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == 1:
        final_tensors = [b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [1],
            [-3] + [-4] + [1] + [-5]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == N:
        final_tensors = [left_contraction, b.get_B(site-1)]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [-5],
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    return np.squeeze(final_contraction)  

In [86]:
derivata = tensorial_derivative(psi=psi,b=b, site=9)
print(derivata.shape)

(3, 3)


In [87]:
def Greedy(A: np.ndarray) -> np.ndarray:
    """
    Greedy compression of the rows of A.

    Parameters
    ----------
    A : np.ndarray
        Input matrix (2D).

    Returns
    -------
    np.ndarray
        Matrix L encoding the greedy merges/deletions of rows of A.
    """
    if A.ndim != 2:
        raise ValueError("A deve essere una matrice 2D")

    M = A.copy()
    L = np.identity(M.shape[0])

    while L.shape[1] > A.shape[1]:
        n_rows = M.shape[0]

        # Row-wise positive sums
        row_sums = np.zeros(n_rows)
        for i in range(n_rows):
            row = M[i]
            row_sums[i] = row[row > 0].sum()

        # Gain from merging each pair of rows
        merge_gain = np.full((n_rows, n_rows), -np.inf, dtype=float)
        for i in range(n_rows):
            for j in range(n_rows):
                if i == j:
                    continue
                merged_row = M[i] + M[j]
                pos_sum = merged_row[merged_row > 0].sum()
                merge_gain[i, j] = pos_sum - row_sums[i] - row_sums[j]

        # Greedy choice: best row to drop vs best pair to merge
        ropt = np.argmin(row_sums)
        iopt, jopt = np.unravel_index(np.argmax(merge_gain), merge_gain.shape)

        if merge_gain[iopt, jopt] > -row_sums[ropt]:
            # Merge rows iopt and jopt
            M[iopt] = M[iopt] + M[jopt]
            M = np.delete(M, jopt, axis=0)

            L[:, iopt] = L[:, iopt] + L[:, jopt]
            L = np.delete(L, jopt, axis=1)
        else:
            # Drop row ropt
            M = np.delete(M, ropt, axis=0)
            L = np.delete(L, ropt, axis=1)

    return L

In [ ]:
def SweepingAlgorithm(b, d, D, err):
    '''
    Input:
        b: lista di tensori
        d: dimensione fisica della Tensor Network
        D: bond dimension della tensor Network di psi
    '''

    # Cominciamo con l'ansatz del tipo A R R ... R
    N = b.L
    psi = build_mps_AR_bond(d=d, N=N, D=D)
    dims = [psi.get_B(i).shape for i in range(psi.L)]
    print(dims)

    # overlap iniziale
    k0 = b.overlap(psi)

    # primo update sul sito 1
    site = 1
    A_tilde = tensorial_derivative(psi=psi, b=b, site=site)
    A_prime = A_tilde.to_ndarray()
    L_out = Greedy(A_prime)
    d_phys, Dr = L_out.shape
    L = L_out.reshape(1, d_phys, Dr)
    L_tenpy = npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR'])
    psi.set_B(site - 1, L_tenpy)

    k1 = b.overlap(psi)
    nstep = 1

    # parametro: ogni quanti update controllare
    K = 1      # oppure N, 2*N, ... a tua scelta
    updates_since_check = 0

    # valore di riferimento per il controllo (ultimo "checkpoint")
    k_ref = k1

    max_steps = 100000  # opzionale, per sicurezza

    while nstep < max_steps:

        # sweep left -> right
        for i in range(N - 1):
            site = i + 1

            # salta il sito 1 solo alla prima iterazione, per non rifare l'update iniziale
            if site == 1 and nstep == 1:
                continue

            A_prime = tensorial_derivative(psi=psi, b=b, site=site).to_ndarray()

            if site > 1 and site < N:
                Dl, d_phys, Dr = A_prime.shape
                A_prime = A_prime.reshape(Dl * d_phys, Dr)
                L = Greedy(A_prime).reshape(Dl, d_phys, Dr)
            else:
                d_phys, Dr = A_prime.shape
                L = Greedy(A_prime).reshape(1, d_phys, Dr)

            L_tenpy = npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR'])
            psi.set_B(i, L_tenpy)

            # nuovo overlap dopo l'update locale
            nstep += 1
            updates_since_check += 1

            if updates_since_check >= K:
                k1 = b.overlap(psi)
                print(k1,nstep)
                # controllo rispetto al valore di riferimento k_ref
                if np.abs(k1 - k_ref) <= err * max(np.abs(k_ref), 1e-12):
                    return psi, k1, nstep
                # non ancora convergente: aggiorno il riferimento
                k_ref = k1
                updates_since_check = 0

        # sweep right -> left
        for j in range(N - 1, 0, -1):
            site = j + 1
            A_prime = tensorial_derivative(psi=psi, b=b, site=site).to_ndarray()


            if site > 1 and site < N:
                Dl, d_phys, Dr = A_prime.shape
                A_prime = A_prime.reshape(Dl, Dr * d_phys)
                R = Greedy(A_prime.T).T.reshape(Dl, d_phys, Dr)
            else:
                Dl, d_phys = A_prime.shape
                R = Greedy(A_prime.T).T.reshape(Dl, d_phys, 1)

            R_tenpy = npc.Array.from_ndarray_trivial(R, labels=['vL', 'p', 'vR'])
            psi.set_B(j, R_tenpy)

            # nuovo overlap
            nstep += 1
            updates_since_check += 1

            if updates_since_check >= K:
                k1 = b.overlap(psi)
                print(k1,nstep)
                if np.abs(k1 - k_ref) <= err * max(np.abs(k_ref), 1e-12):
                    return psi, k1, nstep
                k_ref = k1
                updates_since_check = 0

    # se arrivi qui non hai convergito entro max_steps
    raise RuntimeError("SweepingAlgorithm did not converge within max_steps")

In [209]:
N = 3
B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)
b = build_mps(B_list, d=3)
psi , K, nstep = SweepingAlgorithm(b=b, d=3, D=10, err=1e-6)
print(nstep)

[(1, 3, 3), (3, 3, 3), (3, 3, 1)]
0.6536630000000002 3
0.5240030000000002 5
0.6536630000000002 7
0.5240030000000002 9
0.6536630000000002 11
0.5240030000000002 13
0.6536630000000002 15
0.5240030000000002 17
0.6536630000000002 19
0.5240030000000002 21
0.6536630000000002 23
0.5240030000000002 25
0.6536630000000002 27
0.5240030000000002 29
0.6536630000000002 31
0.5240030000000002 33
0.6536630000000002 35
0.5240030000000002 37
0.6536630000000002 39
0.5240030000000002 41
0.6536630000000002 43
0.5240030000000002 45
0.6536630000000002 47
0.5240030000000002 49
0.6536630000000002 51
0.5240030000000002 53
0.6536630000000002 55
0.5240030000000002 57
0.6536630000000002 59
0.5240030000000002 61
0.6536630000000002 63
0.5240030000000002 65
0.6536630000000002 67
0.5240030000000002 69
0.6536630000000002 71
0.5240030000000002 73
0.6536630000000002 75
0.5240030000000002 77
0.6536630000000002 79
0.5240030000000002 81
0.6536630000000002 83
0.5240030000000002 85
0.6536630000000002 87
0.5240030000000002 89
0.

KeyboardInterrupt: 